# Zadania z ontologii RDF

W tym notebooku znajdują się dwa zadania dotyczące tworzenia i manipulowania ontologiami przy użyciu biblioteki RDFLib.

In [ ]:
from rdflib import Graph, Namespace, Literal, URIRef
from rdflib.namespace import RDF, RDFS, OWL, FOAF, XSD

## Zadanie 1: Stworzenie i przeszukiwanie prostej ontologii RDF

In [ ]:
# Tworzymy nowy graf RDF
g = Graph()

# Definiujemy własną przestrzeń nazw
ex = Namespace("http://example.org/")

# Dodajemy klasy do ontologii
g.add((ex.Animal, RDF.type, RDFS.Class))
g.add((ex.Mammal, RDF.type, RDFS.Class))
g.add((ex.Mammal, RDFS.subClassOf, ex.Animal))
g.add((ex.Dog, RDF.type, RDFS.Class))
g.add((ex.Dog, RDFS.subClassOf, ex.Mammal))

# Dodajemy instancje
g.add((ex.Fido, RDF.type, ex.Dog))
g.add((ex.Fido, FOAF.name, Literal("Fido")))
g.add((ex.Fido, ex.age, Literal(5)))

g.add((ex.Rex, RDF.type, ex.Dog))
g.add((ex.Rex, FOAF.name, Literal("Rex")))
g.add((ex.Rex, ex.age, Literal(3)))

# Wyświetlamy wszystkie trójki w grafie
print("Wszystkie trójki w grafie:")
for s, p, o in g:
    print(s, p, o)

print("\nWszystkie psy w ontologii:")
for dog in g.subjects(RDF.type, ex.Dog):
    name = g.value(dog, FOAF.name)
    age = g.value(dog, ex.age)
    print(f"Imię: {name}, Wiek: {age}")

print("\nWszystkie ssaki w ontologii:")
for mammal in g.subjects(RDFS.subClassOf, ex.Mammal):
    print(mammal)

## Zadanie 2: Rozszerzenie ontologii o relacje i zapytania SPARQL

In [ ]:
# Rozszerzamy ontologię o nowe klasy i relacje
g.add((ex.Person, RDF.type, RDFS.Class))
g.add((ex.owns, RDF.type, RDF.Property))
g.add((ex.owns, RDFS.domain, ex.Person))
g.add((ex.owns, RDFS.range, ex.Dog))

# Dodajemy instancje osób i relacje własności
g.add((ex.John, RDF.type, ex.Person))
g.add((ex.John, FOAF.name, Literal("John")))
g.add((ex.John, ex.owns, ex.Fido))

g.add((ex.Alice, RDF.type, ex.Person))
g.add((ex.Alice, FOAF.name, Literal("Alice")))
g.add((ex.Alice, ex.owns, ex.Rex))

# Wykonujemy zapytanie SPARQL
query = """
PREFIX ex: <http://example.org/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>

SELECT ?person ?personName ?dog ?dogName ?dogAge
WHERE {
    ?person a ex:Person ;
            foaf:name ?personName ;
            ex:owns ?dog .
    ?dog a ex:Dog ;
         foaf:name ?dogName ;
         ex:age ?dogAge .
}
"""

print("Wyniki zapytania SPARQL:")
for row in g.query(query):
    print(f"Osoba: {row.personName}, Pies: {row.dogName}, Wiek psa: {row.dogAge}")

# Zadanie dla użytkownika:
# 1. Dodaj nową klasę 'Cat' jako podklasę 'Mammal'
# 2. Dodaj instancję kota o imieniu "Whiskers" i wieku 2 lata
# 3. Utwórz relację 'likes' między psami a kotami
# 4. Napisz zapytanie SPARQL, które wyświetli wszystkie psy, które lubią koty